# Importing Data / Preprocessing

In [1]:
# Connecting to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [3]:
import re
import string
import gzip
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as pyplot
from pandas import option_context
from sklearn import preprocessing
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.ensemble  import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,classification_report,confusion_matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [6]:
# Reading Data
meta_health = getDF('/content/drive/MyDrive/capstone_project/data_files/meta_Health_and_Personal_Care.json.gz')
meta_health.head(3)

,asin,description,title,imUrl,related,salesRank,categories,price,brand
0,0077614992,This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,0615208479,By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN
2,0615269990,What's wrong with your patient?Do all the symp...,Occam's Razor,http://ecx.images-amazon.com/images/I/51fH-ABe...,"{'also_bought': ['1935660152', '0071743979', '...",{'Toys & Games': 110575},"[[Health & Personal Care, Personal Care, Shavi...",34.99,NaN


In [7]:
# Reading Data
review_health = getDF('/content/drive/MyDrive/capstone_project/data_files/reviews_Health_and_Personal_Care.json.gz')
review_health.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ARMDSTEI0Z7YW,0077614992,dodo,"[0, 0]",This book was a requirement for a college clas...,5.0,great,1360886400,"02 15, 2013"
1,A3FYN0SZYWN74,0615208479,Marilyn Mitzel,"[0, 0]",This is a great gift for anyone who wants to h...,5.0,AMAZING HOW QUICKLY IT WORKS!,1228089600,"12 1, 2008"
2,A2J0WRZSAAHUAP,0615269990,icu-rn,"[0, 0]",Very helpful in learning about different disea...,5.0,Must have,1396742400,"04 6, 2014"


In [8]:
# Merging Data
merge_data = pd.merge(review_health,meta_health,on='asin',how='inner')

In [9]:
# Data Head
merge_data.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,description,title,imUrl,related,salesRank,categories,price,brand
0,ARMDSTEI0Z7YW,0077614992,dodo,"[0, 0]",This book was a requirement for a college clas...,5.0,great,1360886400,"02 15, 2013",This is an example product description.,Principles of Mgmt + Oper-CSUF Custom C,http://ecx.images-amazon.com/images/I/51G%2BRq...,"{'also_bought': ['0471730726', '0132834871', '...",{'Health & Personal Care': 168429},[[Health & Personal Care]],NaN,NaN
1,A3FYN0SZYWN74,0615208479,Marilyn Mitzel,"[0, 0]",This is a great gift for anyone who wants to h...,5.0,AMAZING HOW QUICKLY IT WORKS!,1228089600,"12 1, 2008",By now we all know the benefits of exercise fo...,Brain Fitness Exercises Software,http://ecx.images-amazon.com/images/I/41kbZB04...,NaN,{'Health & Personal Care': 1346973},"[[Health & Personal Care, Personal Care]]",NaN,NaN


In [10]:
# Converting Date Format
merge_data['reviewTime'] = pd.to_datetime(merge_data['reviewTime'])

In [11]:
# Sorting the Data on Time Frame
merge_data = merge_data.sort_values('reviewTime')

In [12]:
# Setting Review Date as Index
merge_data = merge_data.set_index(merge_data['reviewTime'])

In [13]:
# Checking Null Values
merge_data.isnull().sum()

reviewerID             0
asin                   0
reviewerName       31653
helpful                0
reviewText             0
overall                0
summary                0
unixReviewTime         0
reviewTime             0
description       119905
title              11411
imUrl               5278
related           111195
salesRank         141681
categories             0
price             314472
brand             904550
dtype: int64

In [14]:
# Data Shape
merge_data.shape

(2982326, 17)

In [15]:
# Converting Categories Nested List into List
merge_data['categories'] = [i[0] for i in merge_data['categories']]

In [16]:
# Converting Rating Float to Int
merge_data['overall'] = merge_data['overall'].astype('int')

In [17]:
# Info
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2982326 entries, 1998-07-26 to 2014-07-23
Data columns (total 17 columns):
 #   Column          Dtype         
---  ------          -----         
 0   reviewerID      object        
 1   asin            object        
 2   reviewerName    object        
 3   helpful         object        
 4   reviewText      object        
 5   overall         int64         
 6   summary         object        
 7   unixReviewTime  int64         
 8   reviewTime      datetime64[ns]
 9   description     object        
 10  title           object        
 11  imUrl           object        
 12  related         object        
 13  salesRank       object        
 14  categories      object        
 15  price           float64       
 16  brand           object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(13)
memory usage: 409.6+ MB


In [18]:
# Filtering the Records based on Good or Bad
merge_data.insert(6,'sentiment',np.nan)

In [19]:
# Adding Sentiment Columns
lst = []
for i in merge_data['overall']:
  if i > 3:
    lst.append('Positive')
  elif i == 3:
    lst.append('Neutral')
  else:
    lst.append('Negative')

merge_data['sentiment'] = lst

In [20]:
# Filtering the Records based on Helpful or Not
merge_data.insert(4,'help_per',np.nan)

In [21]:
# Adding Values to the Columns helpful percentage
merge_data['help_per'] = [0 if i[1]==0 else round(i[0]/i[1]*100,2) for i in merge_data['helpful']]

In [22]:
# Columns
merge_data.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'help_per',
       'reviewText', 'overall', 'sentiment', 'summary', 'unixReviewTime',
       'reviewTime', 'description', 'title', 'imUrl', 'related', 'salesRank',
       'categories', 'price', 'brand'],
      dtype='object')

In [23]:
# Describe
merge_data.describe()

,help_per,overall,unixReviewTime,price
count,2.982326e+06,2.982326e+06,2.982326e+06,2.667854e+06
mean,3.307806e+01,4.107974e+00,1.358166e+09,2.929901e+01
std,4.361229e+01,1.336448e+00,5.000698e+07,4.011675e+01
min,0.000000e+00,1.000000e+00,9.014112e+08,1.000000e-02
25%,0.000000e+00,4.000000e+00,1.347667e+09,1.013000e+01
50%,0.000000e+00,5.000000e+00,1.372810e+09,1.884000e+01
75%,8.750000e+01,5.000000e+00,1.391472e+09,3.321000e+01
max,2.000000e+02,5.000000e+00,1.406074e+09,9.999900e+02


In [24]:
# Info
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2982326 entries, 1998-07-26 to 2014-07-23
Data columns (total 19 columns):
 #   Column          Dtype         
---  ------          -----         
 0   reviewerID      object        
 1   asin            object        
 2   reviewerName    object        
 3   helpful         object        
 4   help_per        float64       
 5   reviewText      object        
 6   overall         int64         
 7   sentiment       object        
 8   summary         object        
 9   unixReviewTime  int64         
 10  reviewTime      datetime64[ns]
 11  description     object        
 12  title           object        
 13  imUrl           object        
 14  related         object        
 15  salesRank       object        
 16  categories      object        
 17  price           float64       
 18  brand           object        
dtypes: datetime64[ns](1), float64(2), int64(2), object(14)
memory usage: 455.1+ MB


In [25]:
# Shape
merge_data.shape

(2982326, 19)

In [26]:
# Converting (Storing) into CSV
merge_data.to_csv('/content/drive/MyDrive/capstone_project/data_files/merge_data.csv')

In [ ]:
# Copying Data Frame
data = merge_data.copy()

In [ ]:
# Dropping Null Values
merge_data.dropna(inplace=True)

In [ ]:
# Data Shape
merge_data.shape

(1916495, 19)

In [ ]:
# Null Values Check
merge_data.isnull().sum()

reviewerID        0
asin              0
reviewerName      0
helpful           0
help_per          0
reviewText        0
overall           0
sentiment         0
summary           0
unixReviewTime    0
reviewTime        0
description       0
title             0
imUrl             0
related           0
salesRank         0
categories        0
price             0
brand             0
dtype: int64

# Data Preprocessing

## Imputation Part



In [ ]:
# brand "",".","Unknown" value replace with nan

df1["brand"].replace(["",".","Unknown"],np.nan,inplace=True)

In [ ]:
# drop the missing records of title columns

df1.dropna(subset=["title"],inplace=True)

## Brand missing value imputation - Approach:- Impute the brand name whose name are same with first word of title

In [ ]:
df=df1[["title","brand"]]

In [ ]:
df_not_null=df[["title","brand"]][~df.brand.isnull()]
df_null=df[["title","brand"]][df.brand.isnull()]

In [ ]:
df_not_null[["brand","title"]].sample(15)

,brand,title
134145,thinkThin,Crunch Bar- White Chocolate Dipped Mixed Nuts-...
101759,EZ-Access,EZ Access Wheelchair Underneath Carryon Tote -...
72413,Reach,"Reach Total Care Floss Dispensers, Mint, 30 Ya..."
172005,All American EFX,"All American EFX Karbolyn Nutritional Shake, N..."
27184,Trustex,Trustex Strawberry Flavored Premium Latex Cond...
257047,Optimum Nutrition,"Optimum Nutrition Pro Complex Drink Mix, Vanil..."
171351,Amscan,Baby Girl Gingham Gift Bag
163999,smartshake,SmartShake Phil Heath (silver &amp; red) 27oz ...
9236,Solaray,Vitamin B-12 5000mcg Sublingual Solaray 30 Loz...
68987,Parissa,"Parissa- Strip Free Hot Wax (Face and Brow), 4..."


In [ ]:
dic1=dict()

In [ ]:
brand=list(df_not_null["brand"].values)
title=list(df_not_null["title"].values)
brand_title=list(zip(brand,title))

In [ ]:
for i,j in brand_title:
  dic1[i]=j

In [ ]:
dic=dict()
for i ,j in dic1.items():
  len_brand=len(i)
  if i.lower()==j[:len_brand].lower():
    dic[i]=j

In [ ]:
zip_list=list(zip(df_null.index,(df_null["title"].values)))

In [ ]:
for i in dic.keys():
  for indx,j in zip_list:
    if i.lower()==j[:len(i)].lower():
      df1.at[indx,"brand"]=i
    else:
      pass

## Brand missing value imputation - Approach:- Impute the brand name whose name are present in the title

In [ ]:
df_not_null=df[["title","brand"]][~df.brand.isnull()]
df_null=df[["title","brand"]][df.brand.isnull()]

In [ ]:
dic1=dict()

In [ ]:
brand=list(df_not_null["brand"].values)
title=list(df_not_null["title"].values)
brand_title=list(zip(brand,title))

In [ ]:
for i,j in brand_title:
  dic1[i]=j

In [ ]:
dic=dict()
for i ,j in dic1.items():
  len_brand=len(i)
  if i.lower() !=j[:len_brand].lower() and i.lower() in j.lower():
    dic[i]=j

In [ ]:
zip_list=list(zip(df_null.index,(df_null["title"].values)))

In [ ]:
for i in dic.keys():
  for indx,j in zip_list:
    if i.lower() !=j[:len_brand].lower() and i.lower() in j.lower():
      df1.at[indx,"brand"]=i
    else:
      pass

## Price missing value imputation - Approach:-  Price imputation by grouping up the same title and impute by median

In [ ]:
df_null=df1[["title","price"]][df1.price.isnull()]
df_not_null=df1[["title","price"]][~df1.price.isnull()]

In [ ]:
title_unique=df_not_null['title'].unique()

In [ ]:
median=df_not_null.groupby(['title'])[["price"]].median().reset_index()
median

,title,price
0,"# 1 Garcinia Cambogia, Weight Loss Plus Appeti...",21.17
1,# 1 Omega Xl New Zealand - Powerful Omega 3 &q...,59.99
2,# 2 Eye Glasses Non Slip Holders - Grip Specta...,6.98
3,#0000 Steel Wool 4 Pads,10.95
4,#1 &hearts; Garcinia Cambogia Extract &hearts;...,9.97
...,...,...
192089,~Pure Garcinia Cambogia Extract~ 1000MG Garcin...,49.90
192090,~Pure White Kidney Bean Extract!~ 500MG As Rec...,59.97
192091,~Shave Ready~ Shaving Straight Razor 6/8&quot;...,48.99
192092,~Shave Ready~ Stainless Steel Straight Razor 6...,64.99


In [ ]:
dic1=dict()

In [ ]:
price=list(df_not_null["price"].values)
title=list(df_not_null["title"].values)
title_price=list(zip(title,price))

In [ ]:
for i,j in title_price:
  dic1[i]=j

In [ ]:
zip_list=list(zip(df_null.index,(df_null["title"].values)))

In [ ]:
for  i,k in dic1.items():
  for indx,j in zip_list:
    if i == j:
      df1.at[indx,"price"]=k
    else:
      pass

## Price missing value imputation - Approach:- Price imputation by grouping up the same brand and impute by median of those brand

In [ ]:
df_null=df1[["brand","price"]][df1.price.isnull()]
df_not_null=df1[["brand","price"]][~df1.price.isnull()]

In [ ]:
brand_unique=df_not_null['brand'].unique()

In [ ]:
median=df_not_null.groupby(['brand'])[["price"]].median().reset_index()
median

In [ ]:
dic1=dict()

In [ ]:
price=list(df_not_null["price"].values)
brand=list(df_not_null["brand"].values)
brand_price=list(zip(brand,price))

In [ ]:
for i,j in brand_price:
  dic1[i]=j

In [ ]:
zip_list=list(zip(df_null.index,(df_null["brand"].values)))

In [ ]:
for  i,k in dic1.items():
  for indx,j in zip_list:
    if i == j:
      df1.at[indx,"price"]=k
    else:
      pass

In [ ]:
##df1.to_csv("/content/drive/MyDrive/health/final_health_imputed.csv",index=False)

--------------------------------------------------------------------------------

## Review missing value imputation - Approach-impute review name by review id


In [ ]:
df_null1=df1[["reviewerID","reviewerName"]][df1.reviewerName.isnull()]
df_not_null=df1[["reviewerID","reviewerName"]][~df1.reviewerName.isnull()]

In [ ]:
df_not_null.sample(5)
df_null1.head(5)

In [ ]:
dic1=dict()

In [ ]:
reviewerID=list(df_not_null["reviewerID"].values)
reviewerName=list(df_not_null["reviewerName"].values)
reviewerID_reviewerName=list(zip(reviewerID,reviewerName))

In [ ]:
for i,j in reviewerID_reviewerName:
  dic1[i]=j

In [ ]:
df_null=df_null1

In [ ]:
zip_list=list(zip(df_null.index,(df_null["reviewerID"].values)))

In [ ]:
for  i,k in dic1.items():
  for indx,j in zip_list:
    if i == j:
      df1.at[indx,"reviewerName"]=k
    else:
      pass

## Other review name impute by amazon customer

In [ ]:
df1["reviewerName"].value_counts()

In [ ]:
df1["reviewerName"]=df1["reviewerName"].fillna("Amazon Customer")

In [ ]:
# Sampling Data for Sentiment Analysis
group_s=merge_data.groupby(["sentiment"])
sentiments=merge_data.sentiment.unique()
sample_df=pd.DataFrame()
for s in sentiments:
  s_data=group_s.get_group(s)
  sample_data=s_data.sample(frac=0.32,random_state=42)
  sample_df=sample_df.append(sample_data,ignore_index=True)